
# 🖼️ OpenCV Image Processor (Streamlit) — Step‑by‑Step Notebook

**Goal:** This notebook explains the provided Streamlit app, which demonstrates **fundamental Image Processing operations** using **OpenCV**, and shows how each piece works with compact, runnable examples.

You will learn:

- How the GUI is structured in Streamlit (sidebar controls, session state, display area).
- What each image operation does (color conversions, geometric transforms, filtering, enhancement, edge detection, compression).
- How the real-time webcam section is wired.
- How to run, extend, and debug the app.

> This notebook is meant as a **teaching companion** for your Streamlit GUI. It includes runnable demo snippets on synthetic images so you can see effects without relying on external files.



## ✅ Prerequisites & Setup

Install dependencies (in your terminal/command prompt):

```bash
pip install streamlit opencv-python numpy pillow matplotlib
```

Run the app (after you download/write `app.py`):

```bash
streamlit run app.py
```

> **Tip:** If you are on Windows and your webcam doesn't start, close any other app that might be using the camera, and try launching Streamlit from a normal terminal (not inside some IDE). If it still fails, try changing the camera index from `0` → `1` in `cv2.VideoCapture(0)`.



## 🧱 App Architecture (High Level)

- **`ImageProcessor` class**: All core image operations live here (color conversions, transforms, filtering, enhancement, edges).
- **Streamlit UI**:
  - **Sidebar**: Upload image, pick operations, tweak sliders, save output.
  - **Main area**: Side‑by‑side **Original** vs **Processed** + a status bar.
  - **Webcam Bonus**: Optional real‑time processing preview.
- **State**: `st.session_state.operation` and `st.session_state.processor` remember the current operation and images.

**Flow:** Upload → choose operation → app calls the corresponding `ImageProcessor` method → result appears on the right.



## 📎 Full App Code (Reference)

For convenience, the full Streamlit app is embedded below (with small label fixes like `RGB` instead of `RGBO`). You can write it out as `app.py` from the notebook if you want.


In [ ]:

# Optionally write the app code to a local file for you to run:
app_path = "/mnt/data/app.py"
with open(app_path, "w", encoding="utf-8") as f:
    f.write('''
import streamlit as st
import cv2
import numpy as np
from PIL import Image
import io
import time

# Set page configuration
st.set_page_config(
    page_title="OpenCV Image Processor",
    page_icon="🖼️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        font-weight: 600;
        color: #1f2937;
        text-align: center;
        margin-bottom: 2rem;
    }
    .section-header {
        font-size: 1.2rem;
        font-weight: 600;
        color: #374151;
        margin-bottom: 1rem;
        border-bottom: 2px solid #10b981;
        padding-bottom: 0.5rem;
    }
    .image-container {
        border: 2px solid #e5e7eb;
        border-radius: 8px;
        padding: 1rem;
        margin: 1rem 0;
    }
    .stButton > button {
        width: 100%;
        margin: 0.25rem 0;
    }
</style>
""", unsafe_allow_html=True)

class ImageProcessor:
    def __init__(self):
        self.original_image = None
        self.processed_image = None

    def load_image(self, uploaded_file):
        """Load and convert uploaded image to OpenCV format with error handling"""
        if uploaded_file is not None:
            try:
                pil_image = Image.open(uploaded_file)
                if pil_image.mode != 'RGB':
                    pil_image = pil_image.convert('RGB')
                opencv_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
                self.original_image = opencv_image
                return opencv_image
            except Exception as e:
                st.error(f"❌ Failed to load image: {e}")
                return None
        return None

    def convert_for_display(self, image):
        """Convert OpenCV image (BGR) to format suitable for Streamlit display"""
        if len(image.shape) == 3:
            return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    # Color Conversion Operations
    def rgb_to_grayscale(self, image):
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    def rgb_to_hsv(self, image):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        return hsv

    def rgb_to_sepia(self, image):
        sepia_filter = np.array([[0.272, 0.534, 0.131],
                                [0.349, 0.686, 0.168],
                                [0.393, 0.769, 0.189]])
        sepia_img = cv2.transform(image, sepia_filter)
        return np.clip(sepia_img, 0, 255).astype(np.uint8)

    def invert_colors(self, image):
        return cv2.bitwise_not(image)

    # Geometric Transformations
    def rotate_image(self, image, angle):
        height, width = image.shape[:2]
        center = (width // 2, height // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, rotation_matrix, (width, height), 
                                borderValue=(255, 255, 255))
        return rotated

    def scale_image(self, image, scale_factor):
        height, width = image.shape[:2]
        new_width = int(width * scale_factor)
        new_height = int(height * scale_factor)
        scaled = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        canvas = np.full_like(image, 255)
        y_offset = max(0, (height - new_height) // 2)
        x_offset = max(0, (width - new_width) // 2)
        if new_height <= height and new_width <= width:
            canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = scaled
        else:
            crop_y = max(0, (new_height - height) // 2)
            crop_x = max(0, (new_width - width) // 2)
            canvas = scaled[crop_y:crop_y+height, crop_x:crop_x+width]
        return canvas

    def translate_image(self, image, tx, ty):
        height, width = image.shape[:2]
        translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
        translated = cv2.warpAffine(image, translation_matrix, (width, height),
                                   borderValue=(255, 255, 255))
        return translated

    def flip_image(self, image, horizontal=False, vertical=False):
        if horizontal and vertical:
            return cv2.flip(image, -1)
        elif horizontal:
            return cv2.flip(image, 1)
        elif vertical:
            return cv2.flip(image, 0)
        return image

    # Filtering Operations
    def gaussian_blur(self, image, kernel_size):
        if kernel_size % 2 == 0:
            kernel_size += 1
        return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

    def sharpen_image(self, image, strength=1.0):
        kernel = np.array([[0, -strength, 0],
                          [-strength, 1 + 4*strength, -strength],
                          [0, -strength, 0]], dtype=np.float32)
        sharpened = cv2.filter2D(image, -1, kernel)
        return np.clip(sharpened, 0, 255).astype(np.uint8)

    def emboss_effect(self, image):
        kernel = np.array([[-2, -1, 0],
                          [-1, 1, 1],
                          [0, 1, 2]], dtype=np.float32)
        embossed = cv2.filter2D(image, -1, kernel)
        return np.clip(embossed + 128, 0, 255).astype(np.uint8)

    def edge_detection(self, image):
        kernel = np.array([[-1, -1, -1],
                          [-1, 8, -1],
                          [-1, -1, -1]], dtype=np.float32)
        edges = cv2.filter2D(image, -1, kernel)
        return np.clip(edges, 0, 255).astype(np.uint8)

    # Enhancement Operations
    def histogram_equalization(self, image):
        if len(image.shape) == 3:
            yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
            yuv[:, :, 0] = cv2.equalizeHist(yuv[:, :, 0])
            return cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
        else:
            return cv2.equalizeHist(image)

    def contrast_stretch(self, image):
        if len(image.shape) == 3:
            stretched = np.zeros_like(image)
            for i in range(3):
                channel = image[:, :, i]
                min_val, max_val = np.min(channel), np.max(channel)
                if max_val > min_val:
                    stretched[:, :, i] = ((channel - min_val) / (max_val - min_val) * 255).astype(np.uint8)
                else:
                    stretched[:, :, i] = channel
            return stretched
        else:
            min_val, max_val = np.min(image), np.max(image)
            if max_val > min_val:
                return ((image - min_val) / (max_val - min_val) * 255).astype(np.uint8)
            return image

    def adjust_brightness(self, image, brightness):
        adjusted = cv2.add(image, np.ones(image.shape, dtype=np.uint8) * brightness)
        return np.clip(adjusted, 0, 255).astype(np.uint8)

    def gamma_correction(self, image, gamma):
        gamma_corrected = np.power(image / 255.0, gamma) * 255.0
        return np.clip(gamma_corrected, 0, 255).astype(np.uint8)

    # Edge Detection Operations
    def sobel_edge_detection(self, image):
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
        sobel_combined = np.sqrt(sobel_x**2 + sobel_y**2)
        return np.clip(sobel_combined, 0, 255).astype(np.uint8)

    def canny_edge_detection(self, image, low_threshold=50, high_threshold=150):
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        edges = cv2.Canny(gray, low_threshold, high_threshold)
        return edges

    def laplacian_edge_detection(self, image):
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        return np.clip(np.absolute(laplacian), 0, 255).astype(np.uint8)

    def prewitt_edge_detection(self, image):
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        kernel_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.float32)
        kernel_y = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]], dtype=np.float32)
        prewitt_x = cv2.filter2D(gray, cv2.CV_64F, kernel_x)
        prewitt_y = cv2.filter2D(gray, cv2.CV_64F, kernel_y)
        prewitt_combined = np.sqrt(prewitt_x**2 + prewitt_y**2)
        return np.clip(prewitt_combined, 0, 255).astype(np.uint8)


def main():
    st.markdown('<h1 class="main-header">🖼️ OpenCV Image Processor</h1>', unsafe_allow_html=True)

    # Initialize session state
    if 'processor' not in st.session_state:
        st.session_state.processor = ImageProcessor()
    if 'operation' not in st.session_state:
        st.session_state.operation = None

    processor = st.session_state.processor

    # Sidebar for controls
    with st.sidebar:
        st.markdown('<div class="section-header">📁 Image Upload</div>', unsafe_allow_html=True)
        uploaded_file = st.file_uploader("Choose an image file", type=['png', 'jpg', 'jpeg', 'bmp', 'tiff'])

        if uploaded_file is not None:
            original_image = processor.load_image(uploaded_file)
            if original_image is not None:
                st.success(f"✅ Image loaded: {uploaded_file.name}")
                height, width = original_image.shape[:2]
                st.info(f"📏 Dimensions: {width} × {height} pixels")

                # Color Conversions
                st.markdown('<div class="section-header">🎨 Color Conversions</div>', unsafe_allow_html=True)
                col1, col2 = st.columns(2)
                with col1:
                    if st.button("RGB to Grayscale"):
                        st.session_state.operation = "Grayscale"
                        processor.processed_image = processor.rgb_to_grayscale(original_image)
                    if st.button("RGB to Sepia"):
                        st.session_state.operation = "Sepia"
                        processor.processed_image = processor.rgb_to_sepia(original_image)
                with col2:
                    if st.button("RGB to HSV"):
                        st.session_state.operation = "HSV"
                        processor.processed_image = processor.rgb_to_hsv(original_image)
                    if st.button("Invert Colors"):
                        st.session_state.operation = "Invert"
                        processor.processed_image = processor.invert_colors(original_image)

                # Geometric Transformations
                st.markdown('<div class="section-header">🔄 Geometric Transformations</div>', unsafe_allow_html=True)
                st.subheader("🔄 Rotation")
                rotation_angle = st.slider("Rotation Angle", -180, 180, 0, key="rotation")
                if st.button("Apply Rotation"):
                    st.session_state.operation = f"Rotate {rotation_angle}°"
                    processor.processed_image = processor.rotate_image(original_image, rotation_angle)

                st.subheader("📏 Scaling")
                scale_factor = st.slider("Scale Factor", 0.1, 3.0, 1.0, 0.1, key="scale")
                if st.button("Apply Scaling"):
                    st.session_state.operation = f"Scale {scale_factor}x"
                    processor.processed_image = processor.scale_image(original_image, scale_factor)

                st.subheader("↔️ Translation")
                tx = st.slider("Translate X", -100, 100, 0, key="tx")
                ty = st.slider("Translate Y", -100, 100, 0, key="ty")
                if st.button("Apply Translation"):
                    st.session_state.operation = f"Translate ({tx}, {ty})"
                    processor.processed_image = processor.translate_image(original_image, tx, ty)

                col1, col2 = st.columns(2)
                with col1:
                    if st.button("Flip Horizontal"):
                        st.session_state.operation = "Flip Horizontal"
                        processor.processed_image = processor.flip_image(original_image, horizontal=True)
                with col2:
                    if st.button("Flip Vertical"):
                        st.session_state.operation = "Flip Vertical"
                        processor.processed_image = processor.flip_image(original_image, vertical=True)

                # Filtering Operations
                st.markdown('<div class="section-header">🌟 Filtering Operations</div>', unsafe_allow_html=True)
                st.subheader("🌫️ Gaussian Blur")
                blur_strength = st.slider("Blur Strength", 1, 31, 5, 2, key="blur")
                if st.button("Apply Gaussian Blur"):
                    st.session_state.operation = f"Gaussian Blur {blur_strength}"
                    processor.processed_image = processor.gaussian_blur(original_image, blur_strength)

                st.subheader("✨ Sharpen")
                sharpen_strength = st.slider("Sharpen Strength", 0.1, 3.0, 1.0, 0.1, key="sharpen")
                if st.button("Apply Sharpening"):
                    st.session_state.operation = f"Sharpen {sharpen_strength}"
                    processor.processed_image = processor.sharpen_image(original_image, sharpen_strength)

                col1, col2 = st.columns(2)
                with col1:
                    if st.button("Emboss Effect"):
                        st.session_state.operation = "Emboss"
                        processor.processed_image = processor.emboss_effect(original_image)
                with col2:
                    if st.button("Edge Detection"):
                        st.session_state.operation = "Edge Detection"
                        processor.processed_image = processor.edge_detection(original_image)

                # Enhancement Operations
                st.markdown('<div class="section-header">✨ Enhancement Operations</div>', unsafe_allow_html=True)
                col1, col2 = st.columns(2)
                with col1:
                    if st.button("Histogram Equalization"):
                        st.session_state.operation = "Histogram Equalization"
                        processor.processed_image = processor.histogram_equalization(original_image)
                with col2:
                    if st.button("Contrast Stretch"):
                        st.session_state.operation = "Contrast Stretch"
                        processor.processed_image = processor.contrast_stretch(original_image)

                st.subheader("💡 Brightness")
                brightness = st.slider("Brightness", -100, 100, 0, key="brightness")
                if st.button("Apply Brightness"):
                    st.session_state.operation = f"Brightness {brightness}"
                    processor.processed_image = processor.adjust_brightness(original_image, brightness)

                st.subheader("🌈 Gamma Correction")
                gamma = st.slider("Gamma", 0.1, 3.0, 1.0, 0.1, key="gamma")
                if st.button("Apply Gamma Correction"):
                    st.session_state.operation = f"Gamma {gamma}"
                    processor.processed_image = processor.gamma_correction(original_image, gamma)

                # Edge Detection Operations
                st.markdown('<div class="section-header">🔍 Edge Detection</div>', unsafe_allow_html=True)
                col1, col2 = st.columns(2)
                with col1:
                    if st.button("Sobel Edge Detection"):
                        st.session_state.operation = "Sobel"
                        processor.processed_image = processor.sobel_edge_detection(original_image)
                    if st.button("Laplacian Edge Detection"):
                        st.session_state.operation = "Laplacian"
                        processor.processed_image = processor.laplacian_edge_detection(original_image)
                with col2:
                    if st.button("Prewitt Edge Detection"):
                        st.session_state.operation = "Prewitt"
                        processor.processed_image = processor.prewitt_edge_detection(original_image)

                st.subheader("🎯 Canny Edge Detection")
                canny_low = st.slider("Low Threshold", 0, 255, 50, key="canny_low")
                canny_high = st.slider("High Threshold", 0, 255, 150, key="canny_high")
                if st.button("Apply Canny Edge Detection"):
                    st.session_state.operation = f"Canny ({canny_low}, {canny_high})"
                    processor.processed_image = processor.canny_edge_detection(original_image, canny_low, canny_high)

                # Compression & Save
                st.markdown('<div class="section-header">💾 Compression & Save</div>', unsafe_allow_html=True)
                format_comp = st.selectbox("Save Format", ["PNG", "JPG", "BMP"], key="save_format")
                if st.button("💾 Save Processed Image"):
                    if processor.processed_image is not None:
                        if len(processor.processed_image.shape) == 3:
                            pil_image = Image.fromarray(processor.convert_for_display(processor.processed_image))
                        else:
                            pil_image = Image.fromarray(processor.processed_image)

                        buf = io.BytesIO()
                        if format_comp == "JPG":
                            pil_image = pil_image.convert("RGB")
                            pil_image.save(buf, format='JPEG', quality=95)
                        else:
                            pil_image.save(buf, format=format_comp)
                        buf.seek(0)
                        file_size_kb = len(buf.getvalue()) / 1024
                        st.success(f"✅ Saved as {format_comp} - Size: {file_size_kb:.2f} KB")
                        st.download_button(
                            label=f"⬇️ Download {format_comp}",
                            data=buf.getvalue(),
                            file_name=f"processed_{uploaded_file.name.split('.')[0]}.{format_comp.lower()}",
                            mime=f"image/{'jpeg' if format_comp == 'JPG' else format_comp.lower()}"
                        )

                st.markdown("---")
                if st.button("🔄 Reset to Original"):
                    processor.processed_image = None
                    st.session_state.operation = None

    # 🎥 Real-time Webcam (Bonus)
    st.sidebar.markdown('<div class="section-header">🎥 Real-time Webcam (Bonus)</div>', unsafe_allow_html=True)
    run_webcam = st.sidebar.checkbox("▶️ Start Webcam", key="webcam_toggle")
    FRAME_WINDOW = st.empty()

    if run_webcam:
        cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        if not cap.isOpened():
            st.error("❌ Cannot access webcam. Check permissions or close other apps.")
            st.stop()

        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        st.success("✅ Webcam initialized. Press 'Stop' to end.")

        try:
            while run_webcam:
                ret, frame = cap.read()
                if not ret or frame is None or frame.size == 0:
                    st.warning("⚠️ Received empty frame. Skipping...")
                    time.sleep(0.1)
                    continue

                op = st.session_state.operation
                if op:
                    try:
                        if "Grayscale" in op:
                            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                            frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
                        elif "Gaussian Blur" in op:
                            ksize = 5
                            try:
                                ksize = int(op.split()[-1])
                            except Exception:
                                pass
                            if ksize % 2 == 0:
                                ksize += 1
                            frame = cv2.GaussianBlur(frame, (ksize, ksize), 0)
                        elif "Rotate" in op:
                            try:
                                angle = int(op.split()[1].replace("°",""))
                            except Exception:
                                angle = 0
                            h, w = frame.shape[:2]
                            center = (w // 2, h // 2)
                            M = cv2.getRotationMatrix2D(center, angle, 1.0)
                            frame = cv2.warpAffine(frame, M, (w, h))
                        elif "Scale" in op:
                            try:
                                scale = float(op.split()[1].replace("x",""))
                            except Exception:
                                scale = 1.0
                            h, w = frame.shape[:2]
                            new_w, new_h = int(w * scale), int(h * scale)
                            frame = cv2.resize(frame, (new_w, new_h))
                            canvas = np.full((h, w, 3), 255, dtype=np.uint8)
                            y1 = max(0, (h - new_h) // 2)
                            x1 = max(0, (w - new_w) // 2)
                            y2 = min(h, y1 + new_h)
                            x2 = min(w, x1 + new_w)
                            frame_cropped = frame[0:y2-y1, 0:x2-x1]
                            canvas[y1:y2, x1:x2] = frame_cropped
                            frame = canvas
                    except Exception as e:
                        st.warning(f"⚠️ Failed to apply {op}: {e}")

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                FRAME_WINDOW.image(frame_rgb, channels="RGB", use_container_width=True)
                time.sleep(0.03)
                run_webcam = st.session_state.webcam_toggle

        except Exception as e:
            st.error(f"🛑 Critical error: {e}")
        finally:
            cap.release()
            st.info("📹 Webcam released. Ready for next use.")

    # Main display area
    if 'processor' in st.session_state and st.session_state.processor.original_image is not None:
        processor = st.session_state.processor
        uploaded_file_placeholder = st.session_state.get("uploaded_file_name", "uploaded_file")
        col1, col2 = st.columns(2)

        with col1:
            st.markdown('<div class="section-header">📷 Original Image</div>', unsafe_allow_html=True)
            st.image(processor.convert_for_display(processor.original_image), 
                     caption=f"Original", use_container_width=True)

        with col2:
            st.markdown('<div class="section-header">⚙️ Processed Image</div>', unsafe_allow_html=True)
            if processor.processed_image is not None:
                display_image = processor.convert_for_display(processor.processed_image)
                st.image(display_image, caption=f"Applied: {st.session_state.operation}", use_container_width=True)
            else:
                st.info("👈 Select an operation from the sidebar to see the processed result")

        st.markdown("---")
        cols = st.columns(4)
        cols[0].metric("Operation", st.session_state.operation if st.session_state.operation else "None")
        if processor.processed_image is not None:
            h, w = processor.processed_image.shape[:2]
            c = 3 if len(processor.processed_image.shape) == 3 else 1
            cols[1].metric("Dimensions", f"{w}×{h}×{c}")
        else:
            cols[1].metric("Dimensions", "—")
        # File format/size only available for uploads; here we omit for simplicity.
        cols[2].metric("Format", "—")
        cols[3].metric("File Size", "—")

    else:
        st.info("👆 Please upload an image file to get started")
        st.markdown("### 🎯 Available Operations:")
        operations = {
            "🎨 Color Conversions": ["RGB to Grayscale", "RGB to HSV", "RGB to Sepia", "Invert Colors"],
            "🔄 Geometric Transformations": ["Rotation", "Scaling", "Translation", "Flip Horizontal/Vertical"],
            "🌟 Filtering Operations": ["Gaussian Blur", "Sharpen", "Emboss", "Edge Detection"],
            "✨ Enhancement Operations": ["Histogram Equalization", "Contrast Stretch", "Brightness", "Gamma Correction"],
            "🔍 Edge Detection": ["Sobel", "Canny", "Laplacian", "Prewitt"],
            "💾 Compression": ["Save as JPG/PNG/BMP with size comparison"],
            "🎥 Bonus": ["Real-time Webcam Processing"]
        }
        for category, ops in operations.items():
            st.markdown(f"**{category}**")
            st.write(" • ".join(ops))

    st.markdown("---")
    st.caption("📘 Module 1 – Image Processing Fundamentals & Computer Vision")
    st.caption("🛠️ Built with Streamlit + OpenCV | 🟢 Beginner → 🟠 Intermediate → 🔴 Advanced")


if __name__ == "__main__":
    main()
''')
app_path



## 🧪 Quick Demo Setup (Synthetic Image)

To keep this notebook self-contained, we'll create a synthetic test image (gradient + shapes). We'll reuse it across demos below.


In [ ]:

import numpy as np
import cv2
import matplotlib.pyplot as plt

def make_test_image(h=256, w=256):
    # Gradient background
    x = np.linspace(0, 255, w, dtype=np.uint8)
    grad = np.tile(x, (h,1))
    img = np.dstack([grad, np.flipud(grad), np.roll(grad, 64, axis=1)])  # 3-channel RGB-like

    # Draw shapes
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.circle(img_bgr, (w//4, h//3), 30, (255, 0, 0), -1)      # Blue circle (BGR)
    cv2.rectangle(img_bgr, (140, 150), (220, 220), (0, 255, 0), 3)  # Green rectangle
    cv2.putText(img_bgr, "OpenCV", (60, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    return img_bgr

test_img = make_test_image()
plt.figure()
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
plt.title("Synthetic Test Image (BGR shown as RGB)")
plt.axis("off")
plt.show()



## 🎨 Color Conversions (BGR ↔ Gray/HSV/Sepia/Invert)

Your app provides these conversions:

- **BGR → Grayscale**
- **BGR → HSV**
- **Sepia (matrix transform)**
- **Invert**

Below is how each works and a runnable demo.


In [ ]:

# BGR → Gray
gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

# BGR → HSV
hsv = cv2.cvtColor(test_img, cv2.COLOR_BGR2HSV)

# Sepia via 3x3 transform
sepia_filter = np.array([[0.272, 0.534, 0.131],
                         [0.349, 0.686, 0.168],
                         [0.393, 0.769, 0.189]])
sepia = cv2.transform(test_img, sepia_filter).clip(0,255).astype(np.uint8)

# Invert
inverted = cv2.bitwise_not(test_img)

# Show (one plot per figure as requested)
import matplotlib.pyplot as plt

for title, img in [
    ("Grayscale (visualized as RGB)", cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)),
    ("HSV (visualized as RGB conversion)", cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)),
    ("Sepia", cv2.cvtColor(sepia, cv2.COLOR_BGR2RGB)),
    ("Invert", cv2.cvtColor(inverted, cv2.COLOR_BGR2RGB))
]:
    plt.figure()
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")
    plt.show()



## 🔄 Geometric Transformations

The app implements:

- **Rotation:** using `cv2.getRotationMatrix2D` + `cv2.warpAffine`
- **Scaling:** using `cv2.resize` + smart padding/cropping to preserve canvas size
- **Translation:** using an affine matrix `[[1, 0, tx], [0, 1, ty]]`
- **Flips:** via `cv2.flip`

Demo:


In [ ]:

h, w = test_img.shape[:2]

# Rotation
M_rot = cv2.getRotationMatrix2D((w//2, h//2), 30, 1.0)
rotated = cv2.warpAffine(test_img, M_rot, (w, h), borderValue=(255,255,255))

# Scaling (0.6x) with simple resize (no canvas pad here)
scaled = cv2.resize(test_img, (int(w*0.6), int(h*0.6)))

# Translation (+20, -15)
M_trans = np.float32([[1, 0, 20], [0, 1, -15]])
translated = cv2.warpAffine(test_img, M_trans, (w, h), borderValue=(255,255,255))

# Flips
flip_h = cv2.flip(test_img, 1)
flip_v = cv2.flip(test_img, 0)
flip_hv = cv2.flip(test_img, -1)

import matplotlib.pyplot as plt

for title, img in [
    ("Rotated +30°", rotated),
    ("Scaled 0.6x (no canvas)", cv2.cvtColor(scaled, cv2.COLOR_BGR2RGB)),
    ("Translated (+20, -15)", translated),
    ("Flip Horizontal", flip_h),
    ("Flip Vertical", flip_v),
    ("Flip Both", flip_hv)
]:
    plt.figure()
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis("off")
    plt.show()



## 🌟 Filtering

- **Gaussian Blur:** low-pass smoothing; kernel size must be odd.
- **Sharpen:** custom 3×3 kernel emphasizing center pixel while subtracting neighbors.
- **Emboss:** directional gradient-like kernel + bias (shift by 128) to keep values visible.
- **Simple Edge (Laplacian-like kernel):** emphasizes edges by subtracting neighbors.

Demo:


In [ ]:

# Gaussian blur (ksize must be odd)
blur = cv2.GaussianBlur(test_img, (9, 9), 0)

# Sharpen
strength = 1.2
kernel_sharp = np.array([[0, -strength, 0],
                         [-strength, 1 + 4*strength, -strength],
                         [0, -strength, 0]], dtype=np.float32)
sharp = cv2.filter2D(test_img, -1, kernel_sharp).clip(0,255).astype(np.uint8)

# Emboss
kernel_emb = np.array([[-2, -1, 0],
                       [-1, 1, 1],
                       [0, 1, 2]], dtype=np.float32)
emb = cv2.filter2D(test_img, -1, kernel_emb)
emb = np.clip(emb + 128, 0, 255).astype(np.uint8)

# Edge-ish 3x3
kernel_edge = np.array([[-1, -1, -1],
                        [-1, 8, -1],
                        [-1, -1, -1]], dtype=np.float32)
edgeish = cv2.filter2D(test_img, -1, kernel_edge).clip(0,255).astype(np.uint8)

import matplotlib.pyplot as plt

for title, img in [
    ("Gaussian Blur (9x9)", blur),
    ("Sharpen (strength=1.2)", sharp),
    ("Emboss", emb),
    ("Edge-ish (3x3)", edgeish),
]:
    plt.figure()
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis("off")
    plt.show()



## ✨ Enhancement

- **Histogram Equalization (Y channel in YUV):** Increases contrast on luminance only.
- **Contrast Stretching:** Linear scaling channel-wise from min→max to 0→255.
- **Brightness:** Adds a constant to each pixel (clipped to [0,255]).
- **Gamma Correction:** Nonlinear mapping: `output = (input/255)^gamma * 255`.

Demo:


In [ ]:

# Histogram Equalization on Y channel
yuv = cv2.cvtColor(test_img, cv2.COLOR_BGR2YUV)
yuv[:,:,0] = cv2.equalizeHist(yuv[:,:,0])
eq = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)

# Contrast Stretch per channel
cs = np.zeros_like(test_img)
for i in range(3):
    ch = test_img[:,:,i]
    mn, mx = ch.min(), ch.max()
    if mx > mn:
        cs[:,:,i] = ((ch - mn) / (mx - mn) * 255).astype(np.uint8)
    else:
        cs[:,:,i] = ch

# Brightness (+40)
bright = cv2.add(test_img, np.ones(test_img.shape, dtype=np.uint8) * 40).clip(0,255).astype(np.uint8)

# Gamma (0.5 and 2.0)
gamma05 = (np.power(test_img/255.0, 0.5) * 255).clip(0,255).astype(np.uint8)
gamma20 = (np.power(test_img/255.0, 2.0) * 255).clip(0,255).astype(np.uint8)

import matplotlib.pyplot as plt

for title, img in [
    ("Hist. Equalization (Y channel)", eq),
    ("Contrast Stretch", cs),
    ("Brightness +40", bright),
    ("Gamma 0.5 (brighter mids)", gamma05),
    ("Gamma 2.0 (darker mids)", gamma20),
]:
    plt.figure()
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis("off")
    plt.show()



## 🔍 Edge Detection

- **Sobel:** first derivative in X/Y directions; we combine magnitudes.
- **Canny:** multi-stage edge detector with hysteresis thresholds.
- **Laplacian:** second derivative highlighting rapid intensity changes.
- **Prewitt:** similar to Sobel using fixed 3×3 kernels.

Demo:


In [ ]:

# Prepare grayscale
gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

# Sobel
sx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
sy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
sobel = np.sqrt(sx**2 + sy**2)
sobel = np.clip(sobel, 0, 255).astype(np.uint8)

# Canny
canny = cv2.Canny(gray, 50, 150)

# Laplacian
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.clip(np.abs(lap), 0, 255).astype(np.uint8)

# Prewitt
kx = np.array([[-1,0,1],[-1,0,1],[-1,0,1]], dtype=np.float32)
ky = np.array([[-1,-1,-1],[0,0,0],[1,1,1]], dtype=np.float32)
px = cv2.filter2D(gray, cv2.CV_64F, kx)
py = cv2.filter2D(gray, cv2.CV_64F, ky)
prewitt = np.sqrt(px**2 + py**2)
prewitt = np.clip(prewitt, 0, 255).astype(np.uint8)

import matplotlib.pyplot as plt

for title, img in [
    ("Sobel magnitude", cv2.cvtColor(sobel, cv2.COLOR_GRAY2BGR)),
    ("Canny (50,150)", cv2.cvtColor(canny, cv2.COLOR_GRAY2BGR)),
    ("Laplacian |·|", cv2.cvtColor(lap, cv2.COLOR_GRAY2BGR)),
    ("Prewitt magnitude", cv2.cvtColor(prewitt, cv2.COLOR_GRAY2BGR)),
]:
    plt.figure()
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")
    plt.show()



## 💾 Compression & Saving

In the app, users can save processed images as **PNG / JPG / BMP** and see the approximate size.

Below is an example of writing the test image to different formats from Python. (Paths will point to your notebook's `/mnt/data` folder.)


In [ ]:

from PIL import Image
import io, os

# Convert BGR → RGB for PIL
rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
pil = Image.fromarray(rgb)

paths = []
# PNG
buf_png = io.BytesIO()
pil.save(buf_png, format="PNG")
paths.append(("/mnt/data/test_img.png", buf_png))

# JPG (quality=95)
buf_jpg = io.BytesIO()
pil.convert("RGB").save(buf_jpg, format="JPEG", quality=95)
paths.append(("/mnt/data/test_img.jpg", buf_jpg))

# BMP
buf_bmp = io.BytesIO()
pil.save(buf_bmp, format="BMP")
paths.append(("/mnt/data/test_img.bmp", buf_bmp))

for path, buffer in paths:
    with open(path, "wb") as f:
        f.write(buffer.getvalue())

[size for size in [(p, os.path.getsize(p)/1024) for p,_ in paths]]



## 🧠 Streamlit Wiring & Session State

Key patterns used by the app:

- **Session State:**
  - `st.session_state.processor` holds the `ImageProcessor` instance.
  - `st.session_state.operation` stores the last applied operation name (also used by the webcam loop).
- **Sidebar UI:** Buttons/Sliders set `operation` and call the corresponding method.
- **Display:** We show original vs processed using `st.image`.
- **Status Bar:** `st.metric` widgets reflect current operation/dimensions.
- **Webcam:** `st.empty()` creates a live-updating placeholder; loop respects `webcam_toggle`.

> **Common gotcha:** Make sure the webcam UI (checkbox/placeholder) is **outside** tight loops and that you re-read `st.session_state.webcam_toggle` to break the loop when unchecked.



## 🎥 Real-time Webcam Section (Explained)

- Open the camera with `cv2.VideoCapture(0, cv2.CAP_DSHOW)` on Windows (or omit `CV_CAP_DSHOW` elsewhere).
- After reading a frame, the code **optionally applies the current operation** (e.g., grayscale, blur, rotate, scale).
- Convert **BGR → RGB** before showing: `cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)`.
- Use `FRAME_WINDOW.image(...)` to update the UI.
- Sleep a tiny bit (≈0.03s) for ~30 FPS and to let Streamlit refresh.
- Re-check the checkbox: `run_webcam = st.session_state.webcam_toggle` to exit gracefully.

> If the webcam doesn't initialize, show an error and stop. Always `release()` the camera in `finally:` to avoid lock-ups.



## 🧩 Extensions You Can Add

- **More color spaces:** YCbCr, LAB, etc.
- **Morphology:** Dilation, erosion, opening, closing.
- **Affine/Perspective:** 3/4-point warps with draggable landmarks.
- **Split-View Compare:** Slider to reveal original vs processed in one canvas.
- **Batch Mode:** Process multiple images and show a gallery.

Each of these slots nicely into the `ImageProcessor` class + a new sidebar block.



## 🛠️ Troubleshooting Notes

- **"RGBO" labels in your original snippet:** likely a typo — use **"RGB"**.
- **Webcam busy/black screen:** Close other apps; try `VideoCapture(1)`; check browser permissions (if using Streamlit Cloud).
- **Odd kernel sizes:** Gaussian blur kernels must be **odd**; the app ensures this.
- **Color channel order:** OpenCV uses **BGR**, Streamlit expects **RGB**. Always convert before display/saving with PIL.
